In [1]:
import pandas as pd
# read CSV into a pandas dataframe
# download it here:
# https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/YPRQH8
df1 = pd.read_csv('./data/names_dataset.csv',low_memory=False)
df2 = pd.read_csv('./data/wgnd_noctry.csv',low_memory=False)
df3 = pd.read_csv('./data/universalnames_dataset.csv',low_memory=False)

In [2]:
df1 = df1.rename({"sex": "gender"},axis=1)
df3 = df3.rename({"sex": "gender"},axis=1)
df1 = df1.drop(['index'], axis=1)
df3 = df3.drop(['Unnamed: 0'], axis=1)
frames = [df1,df2,df3]
df = pd.concat(frames)
# exclude rows where gender is undefined
df = df[df.gender.isin(['M', 'F'])]
# define input
X = df.name
# define labels (female = 0, male = 1)
y = df.gender
print(df.shape)

(357813, 2)


In [3]:
from sklearn.model_selection import train_test_split
# set aside 10% of the dataset for test
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2)
# inspect resulting shapes
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((286250,), (286250,), (71563,), (71563,))

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
# configure our count vectorizer
vec = CountVectorizer(
    min_df=50,
    ngram_range=(2, 5),
    analyzer='char',
    preprocessor=lambda x: f'^{x.lower().replace(" ", "")}$',
)
# train it
vec.fit(X_train)

CountVectorizer(analyzer='char', min_df=50, ngram_range=(2, 5),
                preprocessor=<function <lambda> at 0x7fb3d8427280>)

In [5]:
# sorted(list(vec.vocabulary_))

In [6]:
# relevant sklearn import
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import decomposition,linear_model
from sklearn.pipeline import Pipeline

vec = CountVectorizer(
    min_df=1,
    ngram_range=(1, 8),
    analyzer='char',
#     preprocessor=lambda x: f'^{x.lower().replace(" ", "")}$',
)

logModel = LogisticRegression(max_iter=300)

# param_grid = [    
#     {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
#     'C' : np.logspace(-4, 4, 20),
#     'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
#     'max_iter' : [100, 1000,2500, 5000]
#     }
# ]

# clf = HalvingGridSearchCV(logModel, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)

pipe = Pipeline(steps=[('vectorizer', vec),('model', logModel)])

pipe.fit(X_train,y_train)

/Users/joannachen/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vectorizer',
                 CountVectorizer(analyzer='char', ngram_range=(1, 8))),
                ('model', LogisticRegression(max_iter=300))])

In [7]:
# some male names
pipe.predict([
    'Liam','Oliver','jianlin','Baek-hyun','Aaditya','Aloïs','Akihiko'
]) # male is True

array(['M', 'M', 'M', 'M', 'M', 'M', 'M'], dtype=object)

In [8]:
# some female names
pipe.predict([
    'Emma','서연','Balam','Aashvi','Andrée','Himari'
]) # female is False

array(['F', 'F', 'M', 'F', 'M', 'F'], dtype=object)

In [9]:
# get raw probabilities instead
pipe.predict_proba([
    'Liam','Oliver','jianlin','Baek-hyun','Aaditya','Aloïs','Akihiko',
    '妍','Emma','Balam','Aashvi','Andrée','Himari'
])

array([[0.10355567, 0.89644433],
       [0.07240551, 0.92759449],
       [0.05853747, 0.94146253],
       [0.13640278, 0.86359722],
       [0.04672119, 0.95327881],
       [0.06812159, 0.93187841],
       [0.01434667, 0.98565333],
       [0.20156697, 0.79843303],
       [0.99717076, 0.00282924],
       [0.02771229, 0.97228771],
       [0.933362  , 0.066638  ],
       [0.16905772, 0.83094228],
       [0.73870121, 0.26129879]])

In [10]:
# relevant sklearn import
from sklearn.metrics import accuracy_score
# predict gender of test names
y_pred1 = pipe.predict(X_train)
y_pred2 = pipe.predict(X_test)
# compute accuracy
print(f'train score: {accuracy_score(y_train, y_pred1)}')
print(f'test score: {accuracy_score(y_test, y_pred2)}')

train score: 0.9790113537117904
test score: 0.937579475427246


In [11]:
# import pickle

# Pkl_Filename = "Pickle_RL_Model.pkl"  

# # save model
# with open(Pkl_Filename, 'wb') as file:  
#     pickle.dump(pipe, file)

In [12]:
# # load model
# with open(Pkl_Filename, 'rb') as file:  
#     Pickled_LR_Model = pickle.load(file)

# Pickled_LR_Model

In [13]:
# # Calculate the Score 
# score = Pickled_LR_Model.score(X_test, y_test)  
# # Print the Score
# print("Test score: {0:.2f} %".format(100 * score))  

In [14]:
# Ypredict = Pickled_LR_Model.predict(['davis'])
# print(Ypredict)